In [1]:
#aisdecoder env
from pyais import decode_msg
import pandas as pd
import pynmea2
from decoder import NMEA_decoder

In [ ]:
file_path = "Input\Vesperino_280921.txt"
ais_msg = NMEA_decoder(file_path)

ais_msg.show_data()





In [2]:
file_path = "Input\\Vesperino_280921.txt"
ais_msg = NMEA_decoder(file_path)
ais_df = ais_msg.to_df()
ais_df.to_excel('Ais_decoded_28_09_21.xlsx')
ais_df

,timestamp,type,repeat,mmsi,status,turn,speed,accuracy,lon,lat,...,band,msg22,partno,vendorid,model,serial,mothership_mmsi,seqno,dest_mmsi,retransmit
0,16:33:19,1,0,247074100,0.0,-128.0,0.0,0.0,14.265167,40.844333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16:33:19,1,0,247058700,0.0,-127.0,27.4,0.0,14.211100,40.789390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16:33:20,1,0,247051400,0.0,-128.0,0.1,1.0,14.255158,40.837328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16:33:22,1,0,247097400,0.0,0.0,26.8,0.0,14.214950,40.793900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16:33:22,3,0,218553000,5.0,0.0,0.0,0.0,14.278117,40.839417,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5585,17:10:48,1,0,247223600,0.0,-128.0,0.0,0.0,14.263300,40.842375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5586,17:10:48,1,0,247058700,0.0,-127.0,27.5,0.0,14.270703,40.816175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5587,17:10:49,3,0,247000400,5.0,0.0,0.0,1.0,14.261082,40.840195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5588,17:10:50,1,0,247048600,0.0,-128.0,0.0,0.0,14.261283,40.842550,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#ENV: py38
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import folium 

style = 'symbol:pin; fill:#0000ff; fill-opacity:0.7; stroke:#ffffff; stroke-opacity:1.0; stroke-width:0.5'
text = None
dateTime = None

In [ ]:
ais = pd.read_excel('Ais_decoded_27_09_21.xlsx')
ais.drop(ais.filter(regex='Unname'), axis=1, inplace=True)
ais=ais[ais['lon'].notnull()]
ais.reset_index(drop=True, inplace=True)
ais

In [ ]:
SHP = gpd.GeoDataFrame(columns = ['style_css', 'label', 'text', 'dateTime', 'geometry'])
for i in range(len(ais)):
    label = ais['mmsi'][i]
    # lat, lon = ais['latitude'][i], ais['longitude'][i]
    lat, lon = float(ais['lat'][i]), float(ais['lon'][i])
    coordinates = [(lon, lat)]
    geometry = Point(coordinates)
    new_row = {'style_css':style, 'label':label, 'text':text, 'dateTime':dateTime, 'geometry':geometry}
    SHP = SHP.append(new_row, ignore_index=True)
# SHP
SHP.to_file('Shapefiles\\Napoli_shp\\test.shp')

In [ ]:
# m = folium.Map(location=[40.7, 14.3], zoom_start=11, tiles='CartoDB positron')

In [ ]:
def plotIcon(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.Marker(location=[point.lat, point.lon],
                        popup=folium.Popup(f"<b>{point.mmsi}</b>", ), #sticky=True, show=True
                        # radius=1,
                        # weight=1,
                        icon=folium.Icon(
                                  icon_color='white',
                                  icon='fa-ship',    # icon code from above link
                                  prefix='fa'),
                        ).add_to(m)


def plotStartIcon(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.Marker(location=[point.lat, point.lon],
                        popup=folium.Popup(f"<b>mmsi:{point.mmsi} time(UTC):{point.timestamp}</b>", ), #sticky=True, show=True
                        # radius=1,
                        # weight=1,
                        icon=folium.Icon(
                                  icon_color='white',
                                  icon='fa-flag',    # icon code from above link
                                  prefix='fa'),
                        ).add_to(m)

def plotEndIcon(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.Marker(location=[point.lat, point.lon],
                        popup=folium.Popup(f"<b>mmsi:{point.mmsi} time(UTC):{point.timestamp}</b>", ), #sticky=True, show=True
                        # radius=1,
                        # weight=1,
                        icon=folium.Icon(
                                  icon_color='white',
                                  icon='fa-ship',    # icon code from above link
                                  prefix='fa'),
                        ).add_to(m)

def plotAnchorIcon(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.Marker(location=[point.lat, point.lon],
                        popup=folium.Popup(f"<b>mmsi:{point.mmsi} time(UTC):{point.timestamp}</b>", ), #sticky=True, show=True
                        # radius=1,
                        # weight=1,
                        icon=folium.Icon(
                                  icon_color='white',
                                  icon='fa-anchor',    # icon code from above link
                                  prefix='fa'),
                        ).add_to(m)


def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.lat, point.lon],
                        popup=folium.Popup(f"<b>mmsi:{point.mmsi}, time(UTC):{point.timestamp}</b>", ), #sticky=True, show=True
                        radius=1,
                        weight=1,
                        fill=True,
                        ).add_to(m)

m = folium.Map(location=[40.7, 14.3], zoom_start=11, tiles='CartoDB positron')
# plotDot(ais.iloc[0])

# ships_mmsi = ais['mmsi'].unique()

# for mmsi in ships_mmsi:
    # ais[ais['mmsi']=mmsi]



# ais.apply(plotDot, axis=1)
# map.fit_bounds(map.get_bounds()) #Set the zoom to the maximum possible
# folium.LayerControl().add_to(m)
# m

In [ ]:
ships_mmsi = ais['mmsi'].unique() # mmsi of ships identified
n_ships_mmsi = ais['mmsi'].nunique() # number of ships identified

def SubsetAis_mmsi(ships_mmsi, ais: pd.DataFrame):
     """
     Subset ais signal to get only the relative to the ship mmsi given in input     
     """
     df = ais[ais['mmsi']==ships_mmsi]
     return df

def PlotPolyline(m: folium.map, SShipAIS: pd.DataFrame):
     """
     Plots a polyline on a map from a list of tuple points
     """
     points = [(x.lat, x.lon) for x in SShipAIS.iloc]
     for point in points: 
          folium.CircleMarker(location=point,
                        radius=0.5,
                        weight=3,
                        fill=True,
                        ).add_to(m)
     if len(points) > 1:
          folium.PolyLine(points, color="black", weight=0.5, opacity=1).add_to(m)
          plotStartIcon(SShipAIS.iloc[0])
          plotEndIcon(SShipAIS.iloc[-1])
     else:
          plotAnchorIcon(SShipAIS.iloc[0])


m = folium.Map(location=[40.75, 14.3], zoom_start=11, tiles='CartoDB positron')
for mmsi in ships_mmsi:
     PlotPolyline(m, SubsetAis_mmsi(mmsi, ais))
m
# df = SubsetAis_mmsi(ships_mmsi[3], ais)
# points = [(x.lat, x.lon) for x in df.iloc]
# points

In [ ]:
m.save('Output\\270921\\map\\map.html')